# Функциональное программирование
    Понятие функционального программирования
    Побочные эффекты
    Функции в Python - граждане первого класса
    Глобальные и локальные переменные
    Вложенные функции и замыкания
    Функции высшего порядка
    Декораторы

### Понятие функционального программирования

Функциональное программирование — парадигма программирования, в которой процесс вычисления трактуется как вычисление значений функций в математическом понимании последних (в отличие от функций как подпрограмм в процедурном программировании). Функциональное программирование предполагает обходиться вычислением результатов функций от исходных данных и результатов других функций, и не предполагает явного хранения состояния программы. Соответственно, не предполагает оно и изменяемости этого состояния.

Основано на Лямбда-исчислении (λ-исчисление), базирующемся на двух операциях: абстрации (операции, позволяющей конструировать функции) и аппликации (операции вызова функции).

Определяет программу как вызов фукнции (аппликация).

Принципы функционального программирования:
* вызов функции для одних и тех же значений параметров должен возвращать одинаковый результат (чистые (pure) функции)
* изменения контекста (внешних переменных) при вызове функции определяется как побочный эффект (нежелателен!)
* изменяемость (mutation) переменных нежелательна
* поддержка функций высших порядков (вызов функций для функций)

### Что делает язык функциональным?

Функции:
* функции "граждане первого класса" (functions as first-class citizens)
* функции высшего порядка (higher-order functions)
* замыкания (closures)
* функции без побочных эффектов (pure functions)
* рекурсия, хвостовая рекурсия (recursion, tail recursion)

Данные:
* неизменяемые структуры данных (immutable data structures)
* вычисление результатов функций вместо явного хранения (и изменения) состояния программы  (avoid changing-state) 

Идиомы:
* итераторы, последовательности, ленивые вычисления, сопоставление с образцом, монады (iterators, sequences, lazy evaluation, pattern matching, monads....

### Поддержка функционального программирования в Python

Реализация:
* функции "граждане первого класса"
* лямбда-функции (анонимные функции)
* поддержка функциональных идиом в стандартной библотеке: map/filter/reduce, itertools, operator
* генераторы могут использоваться для ленивых вычислений (вычислений по требованию)

Недостатки:
* невозможно разделить функции с побочным эффектом и без
* изменяемые переменные
* дорогостоящие операции копирования в памяти
* императивный стиль для циклов
* отсутствие оптимизации для хвостовой рекурсии
* нет синтаксиса для проверки шаблонов
* система типов базируется только на классах
* нет механизма перегрузки функций
* в стандартной библиотеке не реализован механизм построения композиции функций
* имеется только императивный механизм обработки ошибок

In [37]:
# Пример поиска наибольшего делителя в императивном стиле:

def gcd_imp(x, y): 
    r = 0 
    while y > 0: 
        r = x % y 
        print(f'x: {x}, y: {y}, r:{r}')
        x = y 
        y = r 
    return x

In [38]:
# Пример поиска наибольшего делителя в функциональном стиле:

def gcd_fnc(x, y): 
    print(f'x: {x}, y: {y}')
    if y == 0: 
        return x 
    else: 
        return gcd_fnc(y, x % y) # рекурсивный вызов функции

In [39]:
gcd_fnc(9702, 945)

x: 9702, y: 945
x: 945, y: 252
x: 252, y: 189
x: 189, y: 63
x: 63, y: 0


63

In [40]:
factor = 2

In [41]:
def not_pure_f(n):
    return n*factor

In [42]:
not_pure_f(10)

20

In [43]:
factor = 5
not_pure_f(10) # при идентичных параметрах функция возвращает другой результат

50

---

### Побочные эффекты  <a class="anchor" id="побочные"></a>

Изменение контекста (внешних переменных) при вызове функции определяется как побочный эффект.

In [44]:
# Источником побочных эффектов являются использование глобальных переменных:
    
factor = 0 

def multiples(n): 
    global factor 
    factor = factor + 1 
    return factor * n

In [45]:
factor

0

In [46]:
multiples(2)

2

In [47]:
factor

1

In [48]:
multiples(2) # Резальтаты вызова функции с одним и тем же параметром различаются!

4

In [49]:
multiples(2)

6

In [50]:
# Источником побочных переменных также является использование состояния объекта (или функции):

class Multiplier: 
    def __init__(self):
        self.factor = 0 
        
    def multiples(self, n): 
        self.factor = self.factor + 1 
        return self.factor * n

In [51]:
m = Multiplier()

In [52]:
m.multiples(2)

2

In [53]:
m.multiples(2) 

4

In [54]:
m.factor

2

In [55]:
m.multiples(2)

6

In [56]:
# Источником побочных эффектов может быть ввод/вывод: 

def read_byte(f): 
    return f.read(1)

In [57]:
# источником побочных эффектов может быть использование датчика случайных чисел:

import random 

def get_random(n): 
    return random.randrange(1, n + 1)

Отрицательные моменты побочных эффектов:
* тяжелее тестировать программу
* тяжелее производить формальную верефикацию (проверку) программы
* тяжелее создавать программы для параллельных вычислений (параллельной работы на нескольких ядрах / процессорах / компьютерах)


* Можем ли мы писать программы без побочных эффектов? 
    * Или, как минимум, разделять части программы с побочным эффектом и без?

---

## Функции в Python - граждане первого класса <a class="anchor" id="гпк"></a>

Это означает, что функции можно динамически создвать и уничтожать, передвать их в другие функции, возвращать их как значения и так далее.

In [58]:
def add(a, b):
    return a + b

In [59]:
add(2, 3)

5

In [60]:
f = add
f

<function __main__.add(a, b)>

In [61]:
f(2, 3)

5

In [62]:
# Применение: ветвление с использованием словарей:

def insert():
    print('Inserting ...')
    
def delete():
    print('Deleting ...')
    
def update():
    print('Updating ...')    
    
def test():
    print('Testing ...')    

In [63]:
command = input('Введите комнаду (i/d/u/t)')

if command == 'i':
    insert()
elif command == 'd':
    delete()
elif command == 'u':
    update()
elif command == 't':
    test()
else:
    print('Неправильное имя команды.')

Inserting ...


In [64]:
com_d = dict(i=insert, d=delete, u=update, t=test)

command = input('Введите комнаду (i/d/u/t)')
com_d[command]()

Inserting ...


In [65]:
# лямбда функции:
add2 = lambda a, b: a + b

In [66]:
add2

<function __main__.<lambda>(a, b)>

In [67]:
add2(2, 3)

5

In [68]:
import collections

In [69]:
?collections.defaultdict

Init signature: collections.defaultdict(self, /, *args, **kwargs)
Docstring:     
defaultdict(default_factory=None, /, [...]) --> dict with default factory

The default factory is called without arguments to produce
a new value when a key is not present, in __getitem__ only.
A defaultdict compares equal to a dict with the same items.
All remaining arguments are treated the same as if they were
passed to the dict constructor, including keyword arguments.
File:           c:\users\79251\appdata\local\programs\python\python311\lib\collections\__init__.py
Type:           type
Subclasses:     FreezableDefaultDict

In [70]:
other = lambda: print('Неправильное имя команды')

com_dd = collections.defaultdict(lambda: other, com_d.items())

command = input('Введите комнаду (i/d/u/t)')
com_dd[command]()

Inserting ...


Ограничения создания лямбда-функций:
* в теле функции может быть заключено только одно выражение
* значение выражения всегда возвращается, как результат работы функции

### Глобальные и локальные переменные <a class="anchor" id="глобальные-локальные"></a>

Глобальные переменные - это переменные, объявленные в программе вне функции. 
В Pythoп глобальные переменные видны в любой части модуля, включая функции.

In [71]:
def func1(par): 
    print('func begin')
    print(f'\tЗначение глобальной nеременной glob1:{glob1}')     
    glob2 = par 
    print(f'\tЗначение ЛОКАЛЬНОЙ nеременной glob2:{glob2}')     
    print('func end')

In [72]:
glob1 = 10
glob2 = 5

print(f'Значение глобальной nеременной glob1:{glob1}')
print(f'Значение глобальной nеременной glob2:{glob2}')
func1(77)
print(f'Значение глобальной nеременной glob2:{glob2}')

Значение глобальной nеременной glob1:10
Значение глобальной nеременной glob2:5
func begin
	Значение глобальной nеременной glob1:10
	Значение ЛОКАЛЬНОЙ nеременной glob2:77
func end
Значение глобальной nеременной glob2:5


Переменной glob2 внутри функции присваивается значение, переданное при вызове функции.
По этой причине создается новое имя glob2, которое является локальным. 
Все изменения этой переменной внутри функции не затронут значение одноименной глобальной переменной. 

_Локальные переменные_ - это переменные, которым внутри функции присваивается значение.
Если имя локальной переменной совпадает с именем глобальной перемснной, 
то все операции внутри функции осуществляются с локальной переменной, 
а значение глобальной переменной не изменяется. 
Локальные персменные видны только внутри тела функции.

In [73]:
def func2(): 
    print('func begin')
    local = 77 # Локальная переменная 
    glob = 25 # Локальная переменная 
    print(f'\tЗначение ЛОКАЛЬНОЙ nеременной local:{local}')      
    print(f'\tЗначение ЛОКАЛЬНОЙ nеременной glob:{glob}')          
    print('func end')    

In [74]:
glob = 10 # Глобальная переменная 

print(f'Значение глобальной nеременной glob:{glob}')          
func2() # Вызываем функцию 
print(f'Значение глобальной nеременной вне функции glob:{glob}')          

Значение глобальной nеременной glob:10
func begin
	Значение ЛОКАЛЬНОЙ nеременной local:77
	Значение ЛОКАЛЬНОЙ nеременной glob:25
func end
Значение глобальной nеременной вне функции glob:10


In [75]:
print(local) # Ошибка! Попытка обратится к локальной перменной func2 извне ее области видимости

NameError: name 'local' is not defined

Для того чтобы значение глобальной переменной можно было изменить внутри функции, 
необходимо объявить переменную глобальной с помощью ключевого слова global. 

In [76]:
def func2(): 
    print('func begin')
    local = 77 # локальная переменная 
    global glob # объявление глобальной переменной внутри func2
    glob = 25 # ИЗМЕНЕНИЕ ГЛОБАЛЬНОЙ переменной внутри функции
    print(f'\tЗначение локальной nеременной local:{local}')      
    print(f'\tЗначение ГЛОБАЛЬНОЙ nеременной glob:{glob}')          
    print('func end')    

In [77]:
glob = 10 # глобальная переменная 

print(f'Значение глобальной nеременной glob:{glob}')  
func2() # вызов функции 
print(f'Значение глобальной nеременной glob:{glob}')  

Значение глобальной nеременной glob:10
func begin
	Значение локальной nеременной local:77
	Значение ГЛОБАЛЬНОЙ nеременной glob:25
func end
Значение глобальной nеременной glob:25


Поиск идентификатора, используемого внутри функции, будет производиться в следующем порядке: 
1. Поиск объявления идентификатора внутри функции (в локальной области видимости). 
2. Поиск объявления идентификатора в глобальной области. 
3. Поиск во встроенной области видимости (встроенные функции, классы и т. д.).

#### Побочные эффекты

Изменение контекста (внешних переменных) при вызове функции определяется как побочный эффект.

(рассмотрены выше)

Отрицательные моменты побочных эффектов:
* тяжелее тестировать программу
* тяжелее делать программы для параллельных вычислений (параллельной работы на нескольких ядрах / процессорах / компьютерах)
* тяжелее производить формальную верефикацию (проверку) программы

Но, можем ли мы писать программы без побочных эффектов? Или, как минимум, разделять части программы с побочным эффектом и без?

---

## Вложенные функции и замыкания  <a class="anchor" id="вложенные"></a>

В Python одну функцию можно вложить в другую функцию, причем уровень вложенности не ограничен.
В этом случае вложенная функция получает свою собственную локальную область видимости и
имеет достуn к идентификаторам внутри функции-родителя.

Вложенные функции могут создаваться в целях защиты их от всего что происходит вне функции (инкапсуляции):

In [78]:
def outer(num1):
    def inner_add(num2):  # не может быть вызвана вне outer
        return num1 + num2 # используется перменная, объявленная в outer
    num_res = inner_add(2)
    print(num1, num_res)

In [79]:
outer(10)

10 12


In [80]:
inner_add(10) # без специальных действий доступ к внутренней функции извне невозможен

NameError: name 'inner_add' is not defined

Использование внутренних функций для избавления от повторяющихся фрагментов кода в одной функции (применение принципа DRY).

Don’t repeat yourself, DRY (рус. не повторяйся) — это принцип разработки программного обеспечения, нацеленный на снижение повторения информации различного рода, особенно в системах со множеством слоёв абстрагирования. Принцип DRY формулируется как: «Каждая часть знания должна иметь единственное, непротиворечивое и авторитетное представление в рамках системы»

In [81]:
def process(file_name):
    def do_stuff(file_process):
        for line in file_process:
            print(line)
            
    if isinstance(file_name, str):
        with open(file_name, 'r') as f:
            do_stuff(f) # 1е исопльзование вложенной фукнции
    else:
        do_stuff(file_name) #2е использование вложенной функции

Вложенная функция получает свою собственную локальную область видимости и имеет достуn к идентификаторам внутри функции-родителя. Таким образом, помимо локальной, глобальной и встроенной областей видимости добавляется __вложенная область видимости__. При этом поиск идентификаторов вначале производится внутри вложенной функции, затем внутри функции-родителя, далее в функциях более высокого уровня и лишь потом в глобальной и встроенных областях видимости. 

In [82]:
def f_lvl_1(x):
    print('f_lvl_1 begin')
    y = 2
    def f_lvl_2():
        print('\tf_lvl_2 begin')        
        print(f'\tx:{x}')                
#         print(f'\ty:{y}')        
        y = 3 # создается ЛОКАЛЬНАЯ переменная во вложенной функции
        print(f'\ty:{y}')        
        print('\tf_lvl_2 end')                

    print(f'x:{x}')          
    print(f'y:{y}')          
    f_lvl_2()    
    print(f'x:{x}')          
    print(f'y:{y}')          
    print('f_lvl_1 end')                    

In [83]:
f_lvl_1(10)

f_lvl_1 begin
x:10
y:2
	f_lvl_2 begin
	x:10
	y:3
	f_lvl_2 end
x:10
y:2
f_lvl_1 end


In [84]:
def f_lvl_1(x):
    print('f_lvl_1 begin')
    y = 2
    def f_lvl_2():
        print('\tf_lvl_2 begin')        
        print(f'\tx:{x}')                
        print(f'\ty:{y}') # ошибка: обращение к локальной перменной до ее объявления      
        y = 3 # создается ЛОКАЛЬНАЯ переменная во вложенной функции
        print(f'\ty:{y}')        
        print('\tf_lvl_2 end')                

    print(f'x:{x}')          
    print(f'y:{y}')          
    f_lvl_2()    
    print(f'x:{x}')          
    print(f'y:{y}')          
    print('f_lvl_1 end')  

In [85]:
f_lvl_1(10)

f_lvl_1 begin
x:10
y:2
	f_lvl_2 begin
	x:10


UnboundLocalError: cannot access local variable 'y' where it is not associated with a value

In [86]:
def f_lvl_1(x):
    print('f_lvl_1 begin')
    y = 2
    def f_lvl_2():
        print('\tf_lvl_2 begin')        
        print(f'\tx:{x}')    
        nonlocal y # доступ к локальной переменной функции родителя (вложенная область видимости)
        print(f'\ty:{y}') # ошибка: обращение к локальной перменной до ее объявления      
        y = 3 # создается ЛОКАЛЬНАЯ переменная во вложенной функции
        print(f'\ty:{y} (после присовения)')       
        print('\tf_lvl_2 end')                

    print(f'x:{x}')          
    print(f'y:{y}')          
    f_lvl_2()    
    print(f'x:{x}')          
    print(f'y:{y}')          
    print('f_lvl_1 end')          

In [87]:
f_lvl_1(10)

f_lvl_1 begin
x:10
y:2
	f_lvl_2 begin
	x:10
	y:2
	y:3 (после присовения)
	f_lvl_2 end
x:10
y:3
f_lvl_1 end


### Замыкания  <a class="anchor" id="замыкания"></a>

In [88]:
def calculations(a, b):
    def add():
        return a + b
    return add

In [89]:
a = calculations(5, 4)
a

<function __main__.calculations.<locals>.add()>

In [90]:
a()

9

Замыкание (англ. closure) в программировании — функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся её параметрами. Говоря другим языком, замыкание — функция, которая ссылается на свободные переменные в своём контексте.

In [91]:
def generate_power_func(n):
    def nth_power(x):
        return x**n
    return nth_power

In [92]:
n2 = generate_power_func(2)
n2

<function __main__.generate_power_func.<locals>.nth_power(x)>

In [93]:
n2(2), n2(3), n2(4)

(4, 9, 16)

In [94]:
n3 = generate_power_func(3)
n3(2), n3(3), n3(4)

(8, 27, 64)

---

## Функции высшего порядка  <a class="anchor" id="высшего"></a>

Функции высших порядков — это такие функции, которые могут принимать в качестве аргументов и возвращать другие функции.

In [95]:
# функции обратного вызова (callback functions)
def add(a, b):
    return a + b

def sub(a, b):
    return a - b

In [96]:
def apply_operation(operation, arg1, arg2):
    'operation - callback function; arg1, arg2 - arguments'
    return operation(arg1, arg2)

In [97]:
apply_operation(add, 2, 3)

5

In [98]:
apply_operation(sub, 2, 3)

-1

In [99]:
map(str, [1,2,3])

Создание частично подготовленных функций

In [100]:
import functools # модуль с функциями, используемыми в идеомах функционального программирования

In [101]:
help(enumerate)

Help on class enumerate in module builtins:

class enumerate(object)
 |  enumerate(iterable, start=0)
 |  
 |  Return an enumerate object.
 |  
 |    iterable
 |      an object supporting iteration
 |  
 |  The enumerate object yields pairs containing a count (from start, which
 |  defaults to zero) and a value yielded by the iterable argument.
 |  
 |  enumerate is useful for obtaining an indexed list:
 |      (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__(...) from builtins.type
 |      See PEP 585
 |  
 |  --------------------------------------------------------

In [102]:
for i, s in enumerate('abcd'):
    print(s, i)

a 0
b 1
c 2
d 3


In [103]:
enum1 = functools.partial(enumerate, start=5) 
enum1

functools.partial(<class 'enumerate'>, start=5)

In [104]:
lines = list('abcd')
for lino, line in enum1(lines): 
    print(lino, line)

5 a
6 b
7 c
8 d


In [105]:
reader = functools.partial(open, mode="rt", encoding="utf8") 
writer = functools.partial(open, mode="wt", encoding="utf8") 

---

## Декораторы  <a class="anchor" id="декораторы"></a>


Декоратор - это функция, которая принимает функцию или метод в качестве единственного аргумента и  
возвращает новую функцию или метод, включающую декорированную функцию или метод, с дополнительными функциональными возможностями. 

Существуют предопределенные декораторы, например, `@property` и `@classmethod`.

Предположим, что у нас имеется множество функций, выполняющих вычисления, и некоторые из них  
всегда должны возвращать не отрицательный результат. Мы могли бы добавить в каждую из таких функций инструкцию `assert`, но  использование декораторов проще и понятнее. 

In [106]:
def positive_result(function): # декоратор
    def wrapper(*args, **kwargs): # обертка
        result = function(*args, **kwargs) 
        assert result >= 0, function.__name__ + "() result isn't >= 0" 
        return result 
    wrapper.__name__ = function.__name__
    wrapper.__doc__ = function.__doc__ 
    return wrapper 

In [107]:
@positive_result 
def discriminant(a, b, c): 
    return (b ** 2) - (4 * a * c) 

In [108]:
discriminant(1, 1, -1)

5

In [110]:
discriminant(1, 1, 1)

AssertionError: discriminant() result isn't >= 0

In [111]:
@positive_result 
def add3(a, b, c):
    return a + b + c

In [112]:
add3(1,1,-4)

AssertionError: add3() result isn't >= 0

In [113]:
def positive_result2(function): 
# декоратор, который обеспечивает, что wrapper будет носить имя исходной функции и ее строку документирования.     
    @functools.wraps(function) 
    def wrapper(*args, **kwargs): 
        result = function(*args, **kwargs) 
        assert result >= 0, function.__name__ + "() result isn't >= 0 !" 
        return result 
    return wrapper 

In [114]:
@positive_result2 
def discriminant2(a, b, c): 
    return (b ** 2) - (4 * a * c) 

In [115]:
discriminant2(1, 1, 1)

AssertionError: discriminant2() result isn't >= 0 !

Создание декоратора с параметрами

In [116]:
def bounded(minimum, maximum): # внешний генератор декоратора
    def decorator(function): # вунтренний генератор декоратора
        @functools.wraps(function) 
        def wrapper(*args, **kwargs): # декоратор
            result = function(*args, **kwargs) # вызов декарируемой функции
            if result < minimum: 
                return minimum 
            elif result > maximum: 
                return maximum 
            return result 
        return wrapper 
    return decorator

In [117]:
@bounded(0, 100) # обрезает возвращения по нижней и верхней границе 
def percent(amount, total): 
    return (amount / total) * 100 

In [118]:
percent(180, 100)

100

In [119]:
percent(15, 100)

15.0

In [120]:
@bounded(-100, 100) # обрезает возвращения по нижней и верхней границе 
def div2(a, b): 
    return a / b 

In [121]:
div2(-700, 3)

-100